In [1]:
import codecs
from collections import defaultdict
import csv
import string
from stop_words import get_stop_words    # download stop words package from https://pypi.org/project/stop-words/
import numpy as np


stop_words = get_stop_words('english')

admidic=defaultdict(list)
count=0


with open('NOTEEVENTS.csv', 'r') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
     for row in spamreader:
         dischargesummary = row[6].strip()
        
         if dischargesummary in "Discharge Summary":
             admidic[row[1]].append(row[-1].replace('\n',' ').translate(str.maketrans('','',string.punctuation)).lower())
             count=count+1


u=defaultdict(int)
for i in admidic:
    for jj in admidic[i]:
        line=jj.strip('\n').split()
        for j in line:
            u[j]=u[j]+1



u2=defaultdict(int)
for i in u:
        if i.isdigit()==False:
            if u[i]>10:
                if i not in stop_words:
                    u2[i]=u[i]
                    
u=[]   

file1=codecs.open('DIAGNOSES_ICD.csv','r')
ad2c=defaultdict(list)
line=file1.readline()
line=file1.readline()

while line:
    line=line.strip().split(',')

    if line[4][1:-1]!='':
        ad2c[line[2]].append("d_"+line[4][1:-1])
    
    line=file1.readline()



codeu=defaultdict(int)
for i in ad2c:
    for j in ad2c[i]:
        codeu[j]=codeu[j]+1


print(codeu)
cthre=0
fileo=codecs.open("combined_dataset",'w')

IDlist=np.load('IDlist.npy',encoding='bytes').astype(str)
print(IDlist)
for i in IDlist:
    if ad2c[i]!=[]:
        
        fileo.write('start! '+i+'\n')
        fileo.write('codes: ')
        tempc=[]
        for code in ad2c[i]:
            if codeu[code]>=cthre:
                if code[0:5] not in tempc:
                    tempc.append(code[0:5])
       
        for code in tempc:
            fileo.write(code+" ")
        fileo.write('\n')
        fileo.write('notes:\n')
        for line in admidic[i]:    
            thisline=line.strip('\n').split() 
            print(thisline)
            for j in thisline:
                if u2[j]!=0:
                    
                    fileo.write(j+" ")
            fileo.write('\n')
        fileo.write('end!\n')
fileo.close()

In [6]:
while line:
    line=line.strip('\n')
    line=line.split()
    if line[0]=='codes:':
        line=file3.readline()
        line=line.strip('\n')
        line=line.split()
        
        if  line[0]=='notes:':
            tempf=[]
            line=file3.readline()
        
            while line!='end!\n':
                line=line.strip('\n')
                line=line.split()
                for word in line:
                    if word in a3:
                        tempf.append(word)
                
                line=file3.readline()
                
            
            notesdocuments.append(tempf)
    line=file3.readline()

####################################################################################

notesvocab={}
for i in notesdocuments:
    for j in i:
        if j.lower() not in notesvocab:
            notesvocab[j.lower()]=len(notesvocab)
notedata=[]
for i in notesdocuments:
    temp=''
    for j in i:
        temp=temp+j+" "
    notedata.append(temp)
    
print(notesdocuments)
wikidata=[]
for i in wikidocuments:
    temp=''
    for j in i:
        temp=temp+j+" "
    wikidata.append(temp)    
##########################################################
print(notedata)
vect = CountVectorizer(min_df=1,vocabulary=notesvocab,binary=True)
binaryn = vect.fit_transform(notedata)
binaryn=binaryn.A
binaryn=np.array(binaryn,dtype=float)

vect2 = CountVectorizer(min_df=1,vocabulary=notesvocab,binary=True)
binaryk = vect2.fit_transform(wikidata)
binaryk=binaryk.A
binaryk=np.array(binaryk,dtype=float)


np.save('notevec',binaryn)
np.save('wikivec',binaryk)


Defaulting to user installation because normal site-packages is not writeable
  Using cached pyhealth-1.1.3-py2.py3-none-any.whl (113 kB)
     |████████████████████████████████| 77 kB 431 kB/s eta 0:00:01
     |████████████████████████████████| 139.8 MB 1.3 kB/s  eta 0:00:01  |▏                               | 829 kB 800 kB/s eta 0:02:54     |█                               | 4.5 MB 7.9 MB/s eta 0:00:18     |██▊                             | 11.7 MB 469 kB/s eta 0:04:33     |███▋                            | 16.0 MB 330 kB/s eta 0:06:15     |███▋                            | 16.0 MB 330 kB/s eta 0:06:15     |███▊                            | 16.3 MB 330 kB/s eta 0:06:14     |███▉                            | 16.9 MB 330 kB/s eta 0:06:12     |████                            | 17.9 MB 372 kB/s eta 0:05:28            | 19.8 MB 372 kB/s eta 0:05:23     |█████▊                          | 25.0 MB 389 kB/s eta 0:04:55     |██████                          | 26.1 MB 14.9 MB/s eta 0:00:08     |█

In [2]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import codecs


##########################################################
wikivoc={}
codewiki=defaultdict(list)

file2=codecs.open("wikipedia_knowledge",'r','utf-8')
line=file2.readline()
count=0
while line:
    if line[0:4]=='XXXd':
        line=line.strip('\n')
        line=line.split()
        for i in line:
            if i[0:2]=='d_':
                codewiki[i].append(count)
                wikivoc[i]=1
        count=count+1
    line=file2.readline()

################### four codes have two wikidocuments, correct them
codewiki['d_072']=[214]
codewiki['d_698']=[125]
codewiki['d_305']=[250]
codewiki['d_386']=[219]

np.save('wikivoc',wikivoc)
##################################################
filec=codecs.open("combined_dataset",'r','utf-8')

line=filec.readline()

feature=[]
label=[]

while line:
    line=line.strip('\n')
    line=line.split()
    
    if line[0]=='codes:':
        temp=line[1:]
        label.append(temp)
        line=filec.readline()
        line=line.strip('\n')
        line=line.split()
        if  line[0]=='notes:':
            tempf=[]
            line=filec.readline()
           
            while line!='end!\n':
                line=line.strip('\n')
                line=line.split()
                tempf=tempf+line
                line=filec.readline()
            feature.append(tempf)
    line=filec.readline()


prevoc={}
for i in label:
    for j in i:
        if j not in prevoc:
            prevoc[j]=len(prevoc)

##################################
notevec=np.load('notevec.npy')
wikivec=np.load('wikivec.npy')
label_to_ix = {}
ix_to_label={}


for codes in label:
    for code in codes:
        if code not in label_to_ix:
            label_to_ix[code]=len(label_to_ix)
            ix_to_label[label_to_ix[code]]=code

tempwikivec=[]

for i in range(0,len(ix_to_label)):
    if ix_to_label[i] in wikivoc:
        temp=wikivec[codewiki[ix_to_label[i]][0]]
        tempwikivec.append(temp)
    else:
        tempwikivec.append([0.0]*wikivec.shape[1])
wikivec=np.array(tempwikivec)

####################################

data=[]
for i in range(0,len(feature)):
    data.append((feature[i], notevec[i], label[i]))
    
data=np.array(data)  

label_to_ix = {}
ix_to_label={}

for doc, note, codes in data:
    for code in codes:
        if code not in label_to_ix:
            if code in wikivoc:
                label_to_ix[code]=len(label_to_ix)
                ix_to_label[label_to_ix[code]]=code

np.save('label_to_ix',label_to_ix)
np.save('ix_to_label',ix_to_label)

training_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
training_data, val_data = train_test_split(training_data, test_size=0.125, random_state=42)

np.save('training_data',training_data)
np.save('test_data',test_data)
np.save('val_data',val_data)


word_to_ix = {}
ix_to_word={}
ix_to_word[0]='OUT'


for doc, note, codes in training_data:
    for word in doc:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)+1
            ix_to_word[word_to_ix[word]]=word  
    
np.save('word_to_ix',word_to_ix)
np.save('ix_to_word',ix_to_word)

newwikivec=[]
for i in range(0,len(ix_to_label)):
    newwikivec.append(wikivec[prevoc[ix_to_label[i]]])
newwikivec=np.array(newwikivec)
np.save('newwikivec',newwikivec)


Mapping codes: 100%|██████████| 49993/49993 [04:42<00:00, 176.66it/s]


In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix.npy').item()
ix_to_label=np.load('ix_to_label.npy')
training_data=np.load('training_data.npy')
test_data=np.load('test_data.npy')
val_data=np.load('val_data.npy')
word_to_ix=np.load('word_to_ix.npy').item()
ix_to_word=np.load('ix_to_word.npy')
newwikivec=np.load('newwikivec.npy')
wikivoc=np.load('wikivoc.npy').item()

wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))


batchsize=32



def preprocessing(data):

    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    new_data=np.array(new_data)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    new_data=new_data[sortlen]
    
    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
    return batch_data
batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)


######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class CAML(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(CAML, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.embed_drop = nn.Dropout(p=0.2)   
        
        
        self.convs1 = nn.Conv1d(Embeddingsize,300,10,padding=5)
        self.H=nn.Linear(300, tagset_size )   
        self.final = nn.Linear(300, tagset_size)
        
        self.layer2 = nn.Linear(Embeddingsize, 1)
        self.embedding=nn.Linear(rvocsize,Embeddingsize,bias=False)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, vec1, nvec, wiki, simlearning):
        
       
        thisembeddings=self.word_embeddings(vec1)
        thisembeddings = self.embed_drop(thisembeddings)
        thisembeddings=thisembeddings.transpose(1,2)
        
        
        thisembeddings=self.tanh(self.convs1(thisembeddings).transpose(1,2))  
        
        alpha=self.H.weight.matmul(thisembeddings.transpose(1,2))
        alpha=F.softmax(alpha, dim=2)
        
        m=alpha.matmul(thisembeddings)
       
        myfinal=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
       
        if simlearning==1:
            tag_scores = self.sigmoid(myfinal.detach()+vec3)
        else:
            tag_scores = self.sigmoid(myfinal)
        
        
        return tag_scores



topk=10

def trainmodel(model, sim):
    print ('start_training')
    modelsaved=[]
    modelperform=[]
    topk=10
    
    
    bestresults=-1
    bestiter=-1
    for epoch in range(5000):  
        model.train()
        
        lossestrain = []
        recall=[]
        for mysentence in batchtraining_data:
            model.zero_grad()
            
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(),mysentence[1].cuda(),wikivec.cuda(),sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())
        print (epoch)
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print ("XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        model.eval()
    
        recall=[]
        for inputs in batchval_data:
           
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)
    
            loss = loss_function(tag_scores, targets)
            
            targets=targets.data.cpu().numpy()
            tag_scores= tag_scores.data.cpu().numpy()
            
            
            for iii in range(0,len(tag_scores)):
                temp={}
                for iiii in range(0,len(tag_scores[iii])):
                    temp[iiii]=tag_scores[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(targets[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if targets[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)
            
        print ('validation top-',topk, np.mean(recall))
        
        
        
        modelperform.append(np.mean(recall))
        if modelperform[-1]>bestresults:
            bestresults=modelperform[-1]
            bestiter=len(modelperform)-1
        
        if (len(modelperform)-bestiter)>5:
            print (modelperform,bestiter)
            return modelsaved[bestiter]
    
model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

basemodel= trainmodel(model, 0)
torch.save(basemodel, 'CAML_model')

model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
model.load_state_dict(basemodel)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
KSImodel= trainmodel(model, 1)
torch.save(KSImodel, 'KSI_CAML_model')

def testmodel(modelstate, sim):
    model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
    model.cuda()
    model.load_state_dict(modelstate)
    loss_function = nn.BCELoss()
    model.eval()
    recall=[]
    lossestest = []
    
    y_true=[]
    y_scores=[]
    
    
    for inputs in batchtest_data:
       
        targets = inputs[2].cuda()
        
        tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)

        loss = loss_function(tag_scores, targets)
        
        targets=targets.data.cpu().numpy()
        tag_scores= tag_scores.data.cpu().numpy()
        
        
        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)
        
        for iii in range(0,len(tag_scores)):
            temp={}
            for iiii in range(0,len(tag_scores[iii])):
                temp[iiii]=tag_scores[iii][iiii]
            temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop=int(np.sum(targets[iii]))
            hit=0.0
            
            for ii in temp1[0:max(thistop,topk)]:
                if targets[iii][ii[0]]==1.0:
                    hit=hit+1
            if thistop!=0:
                recall.append(hit/thistop)
    y_true=np.concatenate(y_true,axis=0)
    y_scores=np.concatenate(y_scores,axis=0)
    y_true=y_true.T
    y_scores=y_scores.T
    temptrue=[]
    tempscores=[]
    for  col in range(0,len(y_true)):
        if np.sum(y_true[col])!=0:
            temptrue.append(y_true[col])
            tempscores.append(y_scores[col])
    temptrue=np.array(temptrue)
    tempscores=np.array(tempscores)
    y_true=temptrue.T
    y_scores=tempscores.T
    y_pred=(y_scores>0.5).astype(np.int)
    print ('test loss', np.mean(lossestest))
    print ('top-',topk, np.mean(recall))
    print ('macro AUC', roc_auc_score(y_true, y_scores,average='macro'))
    print ('micro AUC', roc_auc_score(y_true, y_scores,average='micro'))
    print ('macro F1', f1_score(y_true, y_pred, average='macro')  )
    print ('micro F1', f1_score(y_true, y_pred, average='micro')  )

print ('CAML alone:           ')
testmodel(basemodel, 0)
print ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print ('KSI+CAML:           ')
testmodel(KSImodel, 1)